In [1]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import urllib.parse
import re
from parce_vacancy import Vacancies
from pymongo import MongoClient
from pymongo import errors
import types

In [2]:
vacancies = Vacancies()
def to_db(self, what: list, where):
    """вставка и обновление полей вакансий"""
    
    duplicates = []
    for element in what:
        element['_id'] = element['link'].split('/')[-1]
        if element['salary_min']:
            element['salary_min'] = int(element['salary_min'])
        if element['salary_max']:
            element['salary_max'] = int(element['salary_max'])
        try:
            where.insert_one(element)
        except errors.DuplicateKeyError:
            where.update_one({"_id": element['_id']}, {'$set': element})
            duplicates.append(element['_id'])
    
    if len(duplicates) > 0:
        print('duplicates were updated')
        
    #возвращает дупликаты
    
vacancies.to_db = types.MethodType(to_db, vacancies)

In [3]:
yourtext = 'python разработчик'
n_pages = 4
site = 'hh'

In [4]:
hh_vacancies = vacancies.find(yourtext, n_pages, site)
len(hh_vacancies)

80

In [5]:
pprint(hh_vacancies[0])


{'address': 'Москва',
 'employer': 'Сбер. IT',
 'link': 'https://hh.ru/vacancy/46222999',
 'name': 'Back-end Developer (Node/Java/Python)',
 'salary_currency': None,
 'salary_max': None,
 'salary_min': None,
 'site': 'hh.ru'}


In [6]:
client = MongoClient('localhost', 27017)
db = client['vacancies']
hh = db.hh

In [7]:
vacancies.to_db(hh_vacancies, hh)

duplicates were updated


In [8]:
#вакансии с большей указанной суммы
number = 100
to_show = {'name': 1, 'salary_min': 1, 'salary_max': 1}
for element in hh.find({'$or': [{'salary_min': { '$gte' : number } }, {'salary_max': { '$gte' : number } }]}, to_show ):
    pprint(element)

{'_id': '46314277',
 'name': 'Python разработчик',
 'salary_max': None,
 'salary_min': 250}
{'_id': '46420684',
 'name': 'Junior Python Developer',
 'salary_max': None,
 'salary_min': 100}
{'_id': '45793420',
 'name': 'Разработчик Python',
 'salary_max': None,
 'salary_min': 180}
{'_id': '46107111',
 'name': 'Программист Python',
 'salary_max': None,
 'salary_min': 100}
{'_id': '45778408',
 'name': 'Backend developer (Python)',
 'salary_max': None,
 'salary_min': 200}
{'_id': '43528617',
 'name': 'Разработчик Python',
 'salary_max': 200,
 'salary_min': None}
{'_id': '46345483',
 'name': 'Ведущий Python разработчик',
 'salary_max': None,
 'salary_min': 300}
{'_id': '45114743',
 'name': 'Senior QA Automation Engineer (Python)',
 'salary_max': None,
 'salary_min': 300}
{'_id': '46411757',
 'name': 'Python разработчик',
 'salary_max': 300,
 'salary_min': None}
{'_id': '46376052',
 'name': 'Программист-разработчик',
 'salary_max': None,
 'salary_min': 100}
{'_id': '45632422',
 'name': 'Midd